In [1]:
import numpy as np
import pandas as pd
import cv2
from sklearn.metrics import accuracy_score
import os
import random
import matplotlib.pyplot as plt
from minisom import MiniSom
from sklearn.preprocessing import minmax_scale
from pylab import plot,axis,show,pcolor,colorbar,bone
from collections import Counter
import matplotlib.gridspec as gridspec
from scipy.spatial import distance
from collections import Counter
from numpy import linalg as LA

In [15]:
samples_train = []
label_train = []
samples_test = []
label_test = []
img_trainsamples = []
dim = (100, 100)
# use mean of each class for initial weight
weights = np.zeros((19, 10000))

i = 0
for filename in os.listdir(f'Dataset/'):
    cnt = 1
    x_c = []
    for image in os.listdir(os.path.join(f'Dataset/',filename)):
        img = cv2.imread(os.path.join(f'Dataset/',filename,image))
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        gray_img = cv2.cvtColor(resized, cv2.COLOR_RGB2GRAY)
#         plt.imshow(gray_img, cmap='gray')
        gray_img_re = gray_img.reshape(-1)
        if cnt <= 80:
            img_trainsamples.append(gray_img)
            samples_train.append(gray_img_re)
            label_train.append(i)
            x_c.append(gray_img_re)
        else:
            samples_test.append(gray_img_re)
            label_test.append(i)
            x_c.append(gray_img_re)
        cnt += 1
    
    weights[i, :] = np.mean(np.array(x_c) / 255)
    i = i + 1
    

x_train = np.array(samples_train) / 255
y_train = np.array(label_train).reshape(-1, 1)   

x_test = np.array(samples_test) / 255
y_test = np.array(label_test).reshape(-1, 1)

In [16]:
T_train = np.zeros((max(y_train.shape), 19))
for i in range(19):
    T_train[y_train.reshape(max(y_train.shape)) == i, i] = 1

T_test = np.zeros((max(y_test.shape), 19))
for i in range(19):
    T_test[y_test.reshape(max(y_test.shape)) == i, i] = 1

In [17]:
# Shuffle
randomize = np.arange(len(x_train))
np.random.shuffle(randomize)
x_train = x_train[randomize]
T_train = T_train[randomize]

T_train_num = np.array(label_train)
T_train_num = T_train_num[randomize]

In [18]:
def label_predict(y_out):
    Y = np.zeros((max(y_out.shape), 19))
    index = np.argmax(y_out, axis=1)
    for i in range(len(y_out)):
        Y[i, index[i]] = 1
    return Y

In [19]:
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

In [51]:
# weights = np.random.uniform(low=0, high=0.0000001, size=(19, 10000))
# weights = x_train[0:19, :]
alpha = 0.5
winner = np.zeros((max(T_train_num.shape), 2))
# largestChange = 0.000000000000000001
# epoch = 1
class_n = []
class_ = [i for i in range(19)]
class_ = np.array(class_)
for epoch in range(100):
    class_n = []
    for i in range(len(x_train)):
        x = x_train[i]
        min1 = distance.euclidean(x, weights[0])
        k = 0
        for j in range(1, 19):
            d = distance.euclidean(x, weights[j])
            if d < min1:
                min1 = d
                k = j
        deltaW = alpha * (x - weights[k])
        weights[k] += deltaW
            
        # winner
        winner[i][0] = T_train_num[i]
        winner[i][1] = k
    alpha = alpha * 0.5
    if epoch == 1 or epoch % 20 == 0:        
        print('epoch:',epoch)
#         alpha = 0.1 * np.exp((-1*epoch)/1000)
        cnt = []
        for j in range(19):
            for i in range(len(x_train)):
                if winner[i][1] == j:
                    cnt.append(winner[i][0])
            if len(cnt) > 0 :
                n = most_frequent(cnt)
                class_n.append(n)
                print('neuron', j, 'class:', n)
            else:
                print('neuron', j, 'class:','none')
            cnt = []
        print('---------------------------------------------------------------')
#     epoch += 1

epoch: 0
neuron 0 class: 18.0
neuron 1 class: 2.0
neuron 2 class: 6.0
neuron 3 class: 2.0
neuron 4 class: 10.0
neuron 5 class: 18.0
neuron 6 class: 8.0
neuron 7 class: 7.0
neuron 8 class: 9.0
neuron 9 class: 18.0
neuron 10 class: 3.0
neuron 11 class: 10.0
neuron 12 class: 14.0
neuron 13 class: 5.0
neuron 14 class: 18.0
neuron 15 class: 5.0
neuron 16 class: 0.0
neuron 17 class: 0.0
neuron 18 class: 11.0
---------------------------------------------------------------
epoch: 1
neuron 0 class: 18.0
neuron 1 class: 2.0
neuron 2 class: 12.0
neuron 3 class: 18.0
neuron 4 class: 18.0
neuron 5 class: 16.0
neuron 6 class: 14.0
neuron 7 class: 7.0
neuron 8 class: 5.0
neuron 9 class: 18.0
neuron 10 class: 3.0
neuron 11 class: 10.0
neuron 12 class: 9.0
neuron 13 class: 5.0
neuron 14 class: 11.0
neuron 15 class: 5.0
neuron 16 class: 8.0
neuron 17 class: 0.0
neuron 18 class: 4.0
---------------------------------------------------------------
epoch: 20
neuron 0 class: 18.0
neuron 1 class: 2.0
neuron 2

In [52]:
cnt = []
c_k = []
for j in range(19):
    for i in range(len(x_train)):
        if winner[i][1] == j:
            cnt.append(winner[i][0])
    if len(cnt) > 0 :
        n = most_frequent(cnt)
        class_n.append(n)
        print('neuron', j, 'class:', n)
    else:
        print('neuron', j, 'class:','none')
    cnt = []
    c_k.append(n)

neuron 0 class: 18.0
neuron 1 class: 2.0
neuron 2 class: 12.0
neuron 3 class: 1.0
neuron 4 class: 18.0
neuron 5 class: 16.0
neuron 6 class: 14.0
neuron 7 class: 7.0
neuron 8 class: 5.0
neuron 9 class: 18.0
neuron 10 class: 3.0
neuron 11 class: 10.0
neuron 12 class: 11.0
neuron 13 class: 5.0
neuron 14 class: 4.0
neuron 15 class: 5.0
neuron 16 class: 2.0
neuron 17 class: 0.0
neuron 18 class: 4.0


In [54]:
def label_predict(x_train, weights, c_k):
    y_pred = []
    for i in range(len(x_train)):
        x = x_train[i]
        min1 = distance.euclidean(x, weights[0])
        k = 0
        for j in range(1, 19):
            d = distance.euclidean(x, weights[j])
            if d < min1:
                min1 = d
                k = j
        y_pred.append(c_k[k])
    return y_pred

In [ ]:
# lvq
weights = w_old
alpha = 0.1
for epoch in range(5):
    for s in range(len(x_train)):
        x = x_train[i]
        min1 = distance.euclidean(x, weights[0])
        k = 0
        for j in range(1, 19):
            d = distance.euclidean(x, weights[j])
            if d < min1:
                min1 = d
                k = j
        if T_train_num[s] == c_k[k]:
            deltaw = alpha * (x - weights[k])
        else:
            deltaw = - alpha * (x - weights[k])
        
        weights[k] += deltaW
    
    y_predict = label_predict(x_train, weights, c_k)
    acc = accuracy_score(y_predict, T_train_num)
    alpha = alpha * 0.5
    print('epoch:', epoch, 'Accuracy:', acc*100)

In [ ]:
# test data
y_predict = label_predict(x_test, weights, c_k)
acc_test = accuracy_score(Y_pred_test, T_test)
print("Accuracy on test data:", acc_test*100)

In [ ]:
# noisy data
def add_noise(samples, ratio):
    row , col = samples[0].shape
    pixels = row * col
    num = int(pixels * ratio)
    noisy_samples = []
#     randomlist = random.sample(range(0, pixels), num)
    for i in range(len(samples)):
        img = samples[i]
        for j in range(num):
            # Pick a random y coordinate
            y_coord = random.randint(0, row - 1)

            # Pick a random x coordinate
            x_coord = random.randint(0, col - 1)

            if img[y_coord][x_coord] == 0:
                img[y_coord][x_coord] = 255

            elif img[y_coord][x_coord] == 255:
                img[y_coord][x_coord] = 0

            else:
                img[y_coord][x_coord] = random.choice([0, 255])

        noisy_samples.append(img)
    return noisy_samples

In [ ]:
noisy_img = add_noise(img_trainsamples, 0.01)

In [ ]:
noisy_samples = []

for i in range(len(noisy_img)):
    noisy_samples.append(noisy_img[i].reshape(-1))

x_noisy = np.array(noisy_samples) / 255

In [ ]:
# test data
y_predict_noisy = label_predict(x_noisy, weights, c_k)
acc_noisy = accuracy_score(y_predict_noisy, T_train)
print("Accuracy on test data:", acc_test*100)